In [1]:
import nltk
# Launch the installer to download "gutenberg" and "stop words" corpora.
#nltk.download()

In [2]:
from textstat.textstat import textstat
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from gensim.models import doc2vec
from collections import namedtuple
from nltk.corpus import stopwords

C:\Users\boyerm\AppData\Local\Continuum\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:

# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?:-[\]]", "", text)
    text = ' '.join(text.split())
    return text

In [7]:
songs = pd.read_csv('C:/Users/boyerm/Documents/Thinkful/songdata.csv')
songs.shape

(57650, 4)

In [6]:
songs

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
5,ABBA,Burning My Bridges,/a/abba/burning+my+bridges_20003011.html,"Well, you hoot and you holler and you make me ..."
6,ABBA,Cassandra,/a/abba/cassandra_20002811.html,Down in the street they're all singing and sho...
7,ABBA,Chiquitita,/a/abba/chiquitita_20002978.html,"Chiquitita, tell me what's wrong \nYou're enc..."
8,ABBA,Crazy World,/a/abba/crazy+world_20003013.html,I was out with the morning sun \nCouldn't sle...
9,ABBA,Crying Over You,/a/abba/crying+over+you_20177611.html,I'm waitin' for you baby \nI'm sitting all al...


In [29]:
eighties_artists = pd.DataFrame(columns= ['artist'])
# maybe other metadata?

In [31]:
artists80 = ['Bon Jovi','Fleetwood Mac','Rod Stewart','Michael Jackson','Rush','Elton John','Bruce Springsteen',
                    'Aerosmith','Paul McCartney','Depeche Mode','Scorpions','David Bowie','Pet Shop Boys','Queen','Cyndi Lauper'
                    ,'Chicago','Judas Priest','Bob Seger','Ozzy Osbourne','Iron Maiden','Paul Simon','Metallica',
                    'Kenny Loggins','John Mellencamp','Journey','Def Leppard','Eurythmics','Genesis','Billy Joel','INXS',
                    'Stevie Wonder','Luther Vandross','U2','ZZ Top','Toto','REO Speedwagon','George Michael','Heart',
                    'Lionel Richie','Phil Collins','Tina Turner','Pink Floyd','Yes','Prince','Pat Benatar','Whitesnake',
                    'Van Halen','Peter Gabriel','Whitney Houston','Sting','Madonna','Foreigner','Tracy Chapman',
                    'Talking Heads','Culture Club','Tears for Fears','Dire Straits','Don Henley','Rem','Men at Work','Wham!']
eighties_artists['artist'] = artists80

In [34]:
eighties_songs = pd.merge(eighties_artists, songs, on='artist', how='inner')
eighties_songs.shape

(7587, 4)

In [35]:
# how many songs from each 80's artist? *make visualization?
songs_per_artist = eighties_songs.groupby('artist').count().sort_values(by = 'song', ascending = False)
print(songs_per_artist)

                   song  link  text
artist                             
Bon Jovi            181   181   181
Fleetwood Mac       180   180   180
Rod Stewart         178   178   178
Michael Jackson     176   176   176
Rush                175   175   175
Bruce Springsteen   175   175   175
Elton John          175   175   175
Aerosmith           171   171   171
Paul McCartney      169   169   169
Depeche Mode        167   167   167
Scorpions           167   167   167
David Bowie         165   165   165
Pet Shop Boys       164   164   164
Queen               163   163   163
Cyndi Lauper        161   161   161
Chicago             160   160   160
Judas Priest        159   159   159
Bob Seger           158   158   158
Ozzy Osbourne       157   157   157
Iron Maiden         156   156   156
Paul Simon          156   156   156
Metallica           155   155   155
Kenny Loggins       153   153   153
John Mellencamp     152   152   152
Journey             150   150   150
Def Leppard         147   14

In [111]:
%%timeit

# make lists with the text and features
artist_name = []
clean_text = []
reading_ease = []
smog_index = []
flesch_kincaid_grade = []
coleman_liau_index = []
readability = []
chall_readability = []
diffwords = []
linsear_write_formula = []
gunning_fog = []
text_standard = []
party1 = []
twords = []
for i in range(0,eighties_songs.shape[0]):  
    z = eighties_songs.artist[i]
    artist_name.append(z)
    x = eighties_songs.text[i]
    clean = text_cleaner(x)
    clean_text.append(clean)    
    
    ease = textstat.flesch_reading_ease(x)
    reading_ease.append(ease)
    smog = textstat.smog_index(x)
    smog_index.append(smog)
    fk_grade = textstat.flesch_kincaid_grade(x)
    flesch_kincaid_grade.append(fk_grade)
    liau = textstat.coleman_liau_index(x)
    coleman_liau_index.append(liau)
    read = textstat.automated_readability_index(x)
    readability.append(read)
    read2 = textstat.dale_chall_readability_score(x)
    chall_readability.append(read2)
    words = textstat.difficult_words(x)
    diffwords.append(words)
    write = textstat.linsear_write_formula(x)
    linsear_write_formula.append(write)
    fog = textstat.gunning_fog(x)
    gunning_fog.append(fog)
    standard = textstat.text_standard(x)
    text_standard.append(standard)
    
    token_words = nltk.word_tokenize(clean)
    twords.append(token_words)

    # could use APPLY here instead of lists to DF

In [112]:
%%timeit

# add features to songs dataframe 
testdf = pd.DataFrame()
testdf['artist_name'] = artist_name
testdf['clean_text'] = clean_text
testdf['reading_ease'] = reading_ease
testdf['smog_index'] = smog_index
testdf['flesch_kincaid_grade'] = flesch_kincaid_grade
testdf['coleman_liau_index'] = coleman_liau_index
testdf['readability'] = readability
testdf['chall_readability'] = chall_readability
testdf['diffwords'] = diffwords
testdf['linsear_write_formula'] = linsear_write_formula
testdf['gunning_fog'] = gunning_fog
testdf['text_standard'] = text_standard
testdf['tokens'] = twords

# eighties_songs['clean_text'] = clean_text
# eighties_songs['reading_ease'] = reading_ease
# eighties_songs['smog_index'] = smog_index
# eighties_songs['flesch_kincaid_grade'] = flesch_kincaid_grade
# eighties_songs['coleman_liau_index'] = coleman_liau_index
# eighties_songs['readability'] = readability
# eighties_songs['chall_readability'] = chall_readability
# eighties_songs['diffwords'] = diffwords
# eighties_songs['linsear_write_formula'] = linsear_write_formula
# eighties_songs['gunning_fog'] = gunning_fog
# eighties_songs['text_standard'] = text_standard
# eighties_songs['tokens'] = twords

In [113]:
testdf.head()

,artist_name,clean_text,reading_ease,smog_index,flesch_kincaid_grade,coleman_liau_index,readability,chall_readability,diffwords,linsear_write_formula,gunning_fog,text_standard,tokens
0,Bon Jovi,Aint no cure for love Aint no cure for love Th...,-9.08,3.1,42.5,7.51,54.1,10.63,30,25.5,50.630296,50th and 51th grade,"[Aint, no, cure, for, love, Aint, no, cure, fo..."
1,Bon Jovi,Feel me like I feel you Won't you let me come ...,-149.45,0.0,96.5,6.65,122.3,16.97,14,51.0,104.631076,104th and 105th grade,"[Feel, me, like, I, feel, you, Wo, n't, you, l..."
2,Bon Jovi,Looking at the pages of my life Faded memories...,-198.51,0.0,113.2,8.10,143.5,20.08,37,51.0,123.485911,50th and 51th grade,"[Looking, at, the, pages, of, my, life, Faded,..."
3,Bon Jovi,Working hard eight days a week Most days we've...,-131.85,0.0,91.8,5.20,116.6,16.68,16,51.0,101.444628,50th and 51th grade,"[Working, hard, eight, days, a, week, Most, da..."
4,Bon Jovi,Some fifteen year old kid sits on his porch Ju...,30.41,11.2,27.4,7.51,34.6,9.22,36,25.5,36.401356,25th and 26th grade,"[Some, fifteen, year, old, kid, sits, on, his,..."


In [114]:
# how long are most songs? # VISUALIZE?
for i in range(0,10):
    print(len(testdf.tokens[i]))

355
299
357
291
333
265
430
294
82
285


In [115]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [116]:
# this picks the most common words in each speech:
allwords = []
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    for word in text:
        word = word.lower()
        word = wordnet_lemmatizer.lemmatize(word)
        if word.isalnum() == True:
            if word not in stopwords.words('english'):
                allwords.append(word)
            else:
                continue
        else:
            continue
            
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(100)]
    

In [117]:
# this goes through the whole song list in the dataframe to find the BOW
all_common_words = []

for i in range(0,testdf.shape[0]):
    z = bag_of_words(testdf.tokens[i])
    all_common_words.append(z)
    z = []
    
# Can flatten list of lists    [[word, word],[word,word]]  ~  sum(list_of_lists, [])

In [118]:
# this finds the common words among all songs top words:

cw=[]

for i in range(0,testdf.shape[0]):
    for word in all_common_words[i]:
        if word not in cw:
            cw.append(word)
        else:
            continue

In [119]:
print(len(cw))  # of unique words

283


In [120]:
print(cw)

['cure', 'aint', 'love', 'need', 'shot', 'come', 'ambulance', 'see', 'said', 'diamond', 'prescription', 'drug', 'find', 'common', 'cold', 'pushing', 'shove', 'cupid', 'must', 'missed', 'went', 'king', 'cash', 'gold', 'hire', 'johnnie', 'cochran', 'late', 'save', 'soul', 'wa', 'blind', 'man', 'mark', 'arrow', 'air', 'hit', 'heart', 'saint', 'valentine', 'whats', 'daddy', 'chapter', 'bird', 'bee', 'could', 'someone', 'call', 'lawyer', 'going', 'shrink', 'found', 'jewelry', 'store', 'buying', 'ring', 'doctor', 'wont', 'help', 'please', 'son', 'sorry', 'terminal', 'disease', 'ca', 'get', 'insurance', 'draw', 'bow', 'waving', 'boy', 'drowning', 'damn', 'good', 'way', 'go', 'aspirin', 'voodoo', 'think', 'hey', 'alive', 'feel', 'like', 'make', 'stay', 'knee', 'rising', 'foot', 'getting', 'higher', 'fly', 'side', 'survived', 'brand', 'new', 'fine', 'let', 'heal', 'day', 'say', 'want', 'hand', 'touch', 'take', 'time', 'look', 'life', 'baby', 'na', 'lovin', 'tonight', 'chorus', 'every', 'somethi

In [121]:
# Ok, now i Have a list of the most common words (unique across all songs). So I want to construct a dataframe that 
# counts how many times the word appears in the speech

In [122]:
wordcount = pd.DataFrame(columns=cw)
wordcount['song_lyrics'] = testdf.clean_text
wordcount.loc[:, cw] = 0

list_of_words = []
for i in range(0,testdf.shape[0]):
    
    for word in testdf.tokens[i]:
        word = word.lower()
        word = wordnet_lemmatizer.lemmatize(word)
        if word.isalnum() == True:
            if word not in stopwords.words('english'):      
                if word in cw:
                    list_of_words.append(word)

                  
    # Populate the row with word counts.
    for w in list_of_words:
        wordcount.loc[i, w] += 1
    
    # reset list again
    list_of_words = []

In [123]:
wordcount

,cure,aint,love,need,shot,come,ambulance,see,said,diamond,...,head,ya,stop,dance,sun,people,fall,end,fire,song_lyrics
0,20,18,17,6,4,4,4,3,3,3,...,0,0,0,0,0,0,0,0,0,Aint no cure for love Aint no cure for love Th...
1,0,0,0,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Feel me like I feel you Won't you let me come ...
2,0,0,0,0,1,1,0,3,0,0,...,0,0,0,0,0,0,0,0,0,Looking at the pages of my life Faded memories...
3,0,0,0,1,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,Working hard eight days a week Most days we've...
4,0,0,0,2,0,2,0,2,2,0,...,0,0,0,0,0,0,0,0,0,Some fifteen year old kid sits on his porch Ju...
5,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,"First time I saw you girl, I knew that it woul..."
6,0,0,6,0,0,0,0,2,0,0,...,0,0,0,0,1,0,0,1,0,"This Romeo is bleeding, but you can't see his ..."
7,0,0,0,2,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,The clock strikes ten Out on the streets again...
8,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,"America the beautiful Oh, beautiful for spacio..."
9,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,1,0,0,0,I used to see you on every tv Your smiling fac...


In [124]:
result = pd.concat([wordcount, testdf], axis=1)
result

,cure,aint,love,need,shot,come,ambulance,see,said,diamond,...,smog_index,flesch_kincaid_grade,coleman_liau_index,readability,chall_readability,diffwords,linsear_write_formula,gunning_fog,text_standard,tokens
0,20,18,17,6,4,4,4,3,3,3,...,3.1,42.5,7.51,54.1,10.63,30,25.500000,50.630296,50th and 51th grade,"[Aint, no, cure, for, love, Aint, no, cure, fo..."
1,0,0,0,0,0,3,0,0,0,0,...,0.0,96.5,6.65,122.3,16.97,14,51.000000,104.631076,104th and 105th grade,"[Feel, me, like, I, feel, you, Wo, n't, you, l..."
2,0,0,0,0,1,1,0,3,0,0,...,0.0,113.2,8.10,143.5,20.08,37,51.000000,123.485911,50th and 51th grade,"[Looking, at, the, pages, of, my, life, Faded,..."
3,0,0,0,1,0,0,0,2,0,0,...,0.0,91.8,5.20,116.6,16.68,16,51.000000,101.444628,50th and 51th grade,"[Working, hard, eight, days, a, week, Most, da..."
4,0,0,0,2,0,2,0,2,2,0,...,11.2,27.4,7.51,34.6,9.22,36,25.500000,36.401356,25th and 26th grade,"[Some, fifteen, year, old, kid, sits, on, his,..."
5,0,0,1,0,0,0,0,0,0,0,...,0.0,87.1,8.21,111.6,16.57,24,51.000000,97.029075,50th and 51th grade,"[First, time, I, saw, you, girl, ,, I, knew, t..."
6,0,0,6,0,0,0,0,2,0,0,...,0.0,141.3,7.52,180.5,23.04,29,51.000000,151.569399,151th and 152th grade,"[This, Romeo, is, bleeding, ,, but, you, ca, n..."
7,0,0,0,2,0,1,0,0,0,0,...,3.1,25.2,6.23,30.8,8.23,21,51.000000,32.356923,50th and 51th grade,"[The, clock, strikes, ten, Out, on, the, stree..."
8,0,0,0,0,0,0,0,1,0,0,...,0.0,32.1,14.01,41.5,11.15,18,44.500000,42.150649,41th and 42th grade,"[America, the, beautiful, Oh, ,, beautiful, fo..."
9,0,0,0,0,0,0,0,2,0,0,...,7.9,5.2,9.10,7.4,5.92,24,6.714286,11.610633,5th and 6th grade,"[I, used, to, see, you, on, every, tv, Your, s..."


In [125]:
result = pd.get_dummies(data=result, columns=['text_standard'])

# Now do the BOW models

In [126]:
# BOW [Random Forest]
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = result['artist_name']
X = np.array(result.drop(['song_lyrics','tokens','clean_text','artist_name'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.5,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.99525441603

Test set score: 0.0529783869267


In [127]:
# BOW with Logistic Regression

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

lr_predicted = lr.predict(X_test)
pd.crosstab(y_test, lr_predicted)

(3793, 494) (3793,)
Training set score: 0.777221196942

Test set score: 0.112018977333


col_0,Aerosmith,Billy Joel,Bob Seger,Bon Jovi,Bruce Springsteen,Chicago,Culture Club,Cyndi Lauper,David Bowie,Def Leppard,...,Tina Turner,Toto,Tracy Chapman,U2,Van Halen,Wham!,Whitesnake,Whitney Houston,Yes,ZZ Top
artist_name,,,,,,,,,,,,,,,,,,,,,
Aerosmith,12,1,6,1,1,2,1,3,1,1,...,2,2,1,1,3,0,0,2,0,4
Billy Joel,2,6,4,2,1,2,0,2,1,1,...,0,1,1,0,1,0,0,0,2,2
Bob Seger,3,2,9,0,1,3,0,5,1,0,...,1,2,1,1,0,0,1,0,0,5
Bon Jovi,1,2,0,16,7,2,0,0,2,2,...,0,1,2,1,1,0,4,0,1,0
Bruce Springsteen,0,3,2,4,23,1,1,2,1,2,...,2,0,1,3,0,0,1,1,0,1
Chicago,4,2,0,1,0,4,0,5,1,1,...,1,5,1,0,1,0,3,2,0,4
Culture Club,1,0,0,1,0,2,6,0,1,0,...,0,0,0,0,0,0,0,0,0,0
Cyndi Lauper,1,1,2,2,1,0,4,5,1,1,...,0,0,1,4,1,0,0,1,3,0
David Bowie,1,0,2,0,0,1,3,1,0,4,...,1,2,1,3,1,0,1,1,1,0


In [128]:
# BOW with Gradient Boosting

clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.996045346691

Test set score: 0.0806536636795
